<a href="https://colab.research.google.com/github/akashbandla/YBI_Internship_Project/blob/main/Movies_Recommendation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# \**Title of the Project**

---
# "Movies Recommendation System Using Collabrative Filtering Technique"




# Objective


---

The objective of the movie recommendation system is to provide personalized movie suggestions to users based on their preferences and past interactions with movies. By analyzing user ratings and behavior, the system aims to predict which movies a user is likely to enjoy and recommend them accordingly. This enhances user experience by offering relevant movie choices, leading to increased user engagement and satisfaction. The system utilizes machine learning algorithms to analyze user data, identify patterns, and make accurate movie recommendations. Ultimately, the goal is to improve user retention and drive user engagement by delivering tailored movie suggestions that align with individual tastes and preferences.










# Data Source

---


*   https://github.com/YBIFoundation/Dataset.git
*   Internet







# #Import Library

In [121]:
!pip install fuzzywuzzy
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

#Import Data


In [ ]:
movies= pd.read_csv("https://github.com/YBIFoundation/Dataset/raw/main/Movies%20Recommendation.csv")
ratings = pd.read_csv("https://github.com/YBIFoundation/Dataset/raw/main/MovieLens.csv")

#Describe Data

In [122]:
movies.head()


,Movie_ID,Movie_Title,Movie_Genre,Movie_Language,Movie_Budget,Movie_Popularity,Movie_Release_Date,Movie_Revenue,Movie_Runtime,Movie_Vote,...,Movie_Homepage,Movie_Keywords,Movie_Overview,Movie_Production_House,Movie_Production_Country,Movie_Spoken_Language,Movie_Tagline,Movie_Cast,Movie_Crew,Movie_Director
0,1,Four Rooms,Crime Comedy,en,4000000,22.876230,09-12-1995,4300000,98.0,6.5,...,NaN,hotel new year's eve witch bet hotel room,It's Ted the Bellhop's first night on the job....,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,"[{'name': 'Allison Anders', 'gender': 1, 'depa...",Allison Anders
1,2,Star Wars,Adventure Action Science Fiction,en,11000000,126.393695,25-05-1977,775398007,121.0,8.1,...,http://www.starwars.com/films/star-wars-episod...,android galaxy hermit death star lightsaber,Princess Leia is captured and held hostage by ...,"[{""name"": ""Lucasfilm"", ""id"": 1}, {""name"": ""Twe...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,"[{'name': 'George Lucas', 'gender': 2, 'depart...",George Lucas
2,3,Finding Nemo,Animation Family,en,94000000,85.688789,30-05-2003,940335536,100.0,7.6,...,http://movies.disney.com/finding-nemo,father son relationship harbor underwater fish...,"Nemo, an adventurous young clownfish, is unexp...","[{""name"": ""Pixar Animation Studios"", ""id"": 3}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
3,4,Forrest Gump,Comedy Drama Romance,en,55000000,138.133331,06-07-1994,677945399,142.0,8.2,...,NaN,vietnam veteran hippie mentally disabled runni...,A man with a low IQ has accomplished great thi...,"[{""name"": ""Paramount Pictures"", ""id"": 4}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,"[{'name': 'Alan Silvestri', 'gender': 2, 'depa...",Robert Zemeckis
4,5,American Beauty,Drama,en,15000000,80.878605,15-09-1999,356296601,122.0,7.9,...,http://www.dreamworks.com/ab/,male nudity female nudity adultery midlife cri...,"Lester Burnham, a depressed suburban father in...","[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes


In [123]:
ratings.head()

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


#Data Preprocessing

In [124]:
data_frame=ratings.pivot(index='movieId',columns='userId',values='rating').fillna(0.0)
data_frame


userId,1,2,3,4,5,6,7,8,9,10,...,7036,7037,7038,7039,7040,7041,7042,7043,7044,7045
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,4.0,3.0,4.0,0.0,0.0,4.0,0.0,3.5,...,0.0,0.0,0.0,5.0,3.5,4.0,4.5,0.0,0.0,4.5
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [125]:
no_users_voted=ratings.groupby('movieId')['rating'].agg('count')

In [126]:
no_movies_voted=ratings.groupby('userId')['rating'].agg('count')

In [127]:
final_dataset=data_frame.loc[no_users_voted[no_users_voted > 10].index,:]

In [128]:
final_dataset=final_dataset.loc[:,no_movies_voted[no_movies_voted > 50].index]
sparsed_data=csr_matrix(final_dataset.values)

In [129]:
final_dataset.reset_index(inplace=True)

#Modeling


In [130]:
knn_model=NearestNeighbors(metric='cosine',algorithm='brute',n_jobs=-1)
knn_model.fit(sparsed_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [147]:
def movies_recommendation(movie_name,n_movies_to_recommand):
  movie_list=movies[movies['Movie_Title'].str.contains(movie_name)]
  try:
    if len(movie_list):
        movie_id= movie_list.iloc[0]['Movie_ID']
        csr_index=final_dataset[final_dataset['movieId'] == movie_id].index[0]
        distances,indices=knn_model.kneighbors(sparsed_data[csr_index],n_neighbors=n_movies_to_recommand+1)
        rec_movies_indices=sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x:x[1])[1:]
        recommended_list=[]
        for val in rec_movies_indices:
          movie_id = final_dataset.iloc[val[0]]['movieId']
          id = movies[movies['Movie_ID'] == movie_id].index
          recommended_list.append({'Title':movies.iloc[id]['Movie_Title'].values[0],'Movie_Genre':movies.iloc[id]['Movie_Genre'].values[0],'Movie_Language':movies.iloc[id]['Movie_Language'].values[0]})
        rec_df=pd.DataFrame(recommended_list,index=range(1,n_movies_to_recommand+1))
        return rec_df
    else:
        print('Enter Another Movie name')
  except TypeError:
    print("This movie is not qualified for this recommendation model! Please try again With another movie")

In [148]:
input_name=input("Enter any Movie name\n")
movie_name=input_name.title()
n_movies_to_recommand=int(input("Enter the number of movies that you want as recommandation :\n"))
movies_recommendation(movie_name,n_movies_to_recommand)

Enter any Movie name
the avengers
Enter the number of movies that you want as recommandation :
10


,Title,Movie_Genre,Movie_Language
1,Loving Annabelle,Drama Romance,en
2,Mindhunters,Mystery Thriller Crime,en
3,The Beyond,Horror,it
4,Bulletproof Monk,Action Comedy Fantasy,en
5,The Singles Ward,Comedy Drama Family Romance,en
6,Damnation Alley,Action Adventure Science Fiction,en
7,Life or Something Like It,Comedy Drama Romance,en
8,Lage Raho Munna Bhai,Comedy Drama Romance,en
9,Travellers and Magicians,Adventure Drama Foreign,en
10,The Other Side of Heaven,Action Adventure Drama Family,en
